In [1]:
import requests
import itertools
import pandas as pd
from bs4 import BeautifulSoup

### Пример одной ссылки – одного заседания

In [2]:
url = "http://transcript.duma.gov.ru/node/4680/"

In [3]:
# забираем данные со страницы

page = requests.get(url)
soup = BeautifulSoup(page.text)

In [4]:
# session_name: название заседания – будет служить названием Excel-файла
# div: раздел с текстом протокола
# parts: все абзацы протокола

session_name = soup.find("h1").text
div = soup.find("div", {"id" : "selectable-content"})
parts = div.find_all("p")

In [5]:
# находим индексы элементов, где есть полужирный шрифт
# полужирным выделены имена выступающих

indices = [0]
for i, part in enumerate(parts):
    if part.find("b") is not None:
        indices.append(i)

In [6]:
# создаем пары последовательных индексов,
# чтобы сегментировать список абзацев,
# объединить абзацы между именами выступающих, 
# например, с 1 по 12, с 12 по 26 итд

a, b = itertools.tee(indices)
next(b, None)
res = list(zip(a, b))

In [7]:
# segments: список списков
# в каждом списке в segments на первом месте – имя, 
# далее – текст (с именем вместе, там везде по-разному может обрезаться)

segments = []
for r in res:
    segments.append(parts[r[0]:r[1]])    

In [8]:
# filtered: отбираем все списки длины более 1
#  чтобы убрать лишние технические заметки

filtered = list(filter(lambda x : len(x) > 1, segments))

In [9]:
# pairs: список пар, на первом месте имя (имена на этом этапе не уникальны),
# на втором – слова выступающих

# names – список уникальный имен – фамилия и инициалы, без партий и проч
# отсортирован по алфавиту

pairs = [(f[0].find("b").text, "".join([t.text for t in f[0:]])) for f in filtered]
names = sorted(list(set([p[0].strip(",") for p in pairs])))

In [10]:
# создаем словарь data, ключи – уникальные имена
# к значениям в цикле добавляем фрагменты речи

data = dict(zip(names, [""] * len(names)))

for name in names:
    for p1, p2 in pairs:
        if p1.startswith(name):
            data[name] += p2

In [11]:
# преобразуем словарь в датафрейм

df = pd.DataFrame(data.items(), columns = ["name", "words"])

### Общий случай

In [12]:
# пишем функцию на основе кода выше
# на входе – ссылка на страницу с заседанием
# на выходе – в файл Excel сохраняется датафрейм,
# название файла = дата заседания
# одна строка датафрейма = один выступающий

def get_words(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text)
    session_name = soup.find("h1").text
    div = soup.find("div", {"id" : "selectable-content"})
    parts = div.find_all("p")
    
    indices = [0]
    for i, part in enumerate(parts):
        if part.find("b") is not None:
            indices.append(i)
            
    a, b = itertools.tee(indices)
    next(b, None)
    res = list(zip(a, b))
    
    segments = []
    for r in res:
        segments.append(parts[r[0]:r[1]])

    filtered = list(filter(lambda x : len(x) > 1, segments))
    pairs = [(f[0].find("b").text, "".join([t.text for t in f[0:]])) for f in filtered]
    names = sorted(list(set([p[0].strip(",") for p in pairs])))
    
    data = dict(zip(names, [""] * len(names)))

    for name in names:
        for p1, p2 in pairs:
            if p1.startswith(name):
                data[name] += p2
                
    df = pd.DataFrame(data.items(), columns = ["name", "words"])
    df.to_excel(session_name + ".xlsx", encoding ='UTF-8')

In [13]:
# проверяем на новой ссылке

get_words("http://transcript.duma.gov.ru/node/4587/")

In [14]:
#1-ая сессия ГД седьмого созыва (осень 2016 г.)
urls_0 = ['http://transcript.duma.gov.ru/node/4521/', 'http://transcript.duma.gov.ru/node/4565/', 'http://transcript.duma.gov.ru/node/4524/',
         'http://transcript.duma.gov.ru/node/4553/', 'http://transcript.duma.gov.ru/node/4556/', 'http://transcript.duma.gov.ru/node/4562/',
         'http://transcript.duma.gov.ru/node/4571/', 'http://transcript.duma.gov.ru/node/4527/', 'http://transcript.duma.gov.ru/node/4530/',
         'http://transcript.duma.gov.ru/node/4534/', 'http://transcript.duma.gov.ru/node/4538/', 'http://transcript.duma.gov.ru/node/4541/',
         'http://transcript.duma.gov.ru/node/4544/', 'http://transcript.duma.gov.ru/node/4547/', 'http://transcript.duma.gov.ru/node/4550/',
         'http://transcript.duma.gov.ru/node/4558/', 'http://transcript.duma.gov.ru/node/4568/']

In [15]:
#2-ая сессия ГД седьмого созыва (весна 2017 г.)
urls_1 = ['http://transcript.duma.gov.ru/node/4581/', 'http://transcript.duma.gov.ru/node/4597/', 'http://transcript.duma.gov.ru/node/4667/',
       'http://transcript.duma.gov.ru/node/4680/', 'http://transcript.duma.gov.ru/node/4699/', 'http://transcript.duma.gov.ru/node/4715/',
       'http://transcript.duma.gov.ru/node/4576/', 'http://transcript.duma.gov.ru/node/4579/', 'http://transcript.duma.gov.ru/node/4595/',
       'http://transcript.duma.gov.ru/node/4618/', 'http://transcript.duma.gov.ru/node/4624/', 'http://transcript.duma.gov.ru/node/4626/',
       'http://transcript.duma.gov.ru/node/4629/', 'http://transcript.duma.gov.ru/node/4665/', 'http://transcript.duma.gov.ru/node/4665/',
       'http://transcript.duma.gov.ru/node/4672/', 'http://transcript.duma.gov.ru/node/4677/', 'http://transcript.duma.gov.ru/node/4704/',
       'http://transcript.duma.gov.ru/node/4708/', 'http://transcript.duma.gov.ru/node/4584/', 'http://transcript.duma.gov.ru/node/4587/',
       'http://transcript.duma.gov.ru/node/4593/', 'http://transcript.duma.gov.ru/node/4600/', 'http://transcript.duma.gov.ru/node/4606/',
       'http://transcript.duma.gov.ru/node/4610/', 'http://transcript.duma.gov.ru/node/4611/', 'http://transcript.duma.gov.ru/node/4615/',
       'http://transcript.duma.gov.ru/node/4620/', 'http://transcript.duma.gov.ru/node/4638/', 'http://transcript.duma.gov.ru/node/4645/']

In [16]:
#3-ая сессия ГД седьмого созыва (осень 2017 г.)
urls_2 = ['http://transcript.duma.gov.ru/node/4769/', 'http://transcript.duma.gov.ru/node/4766/', 'http://transcript.duma.gov.ru/node/4760/',
         'http://transcript.duma.gov.ru/node/4778/', 'http://transcript.duma.gov.ru/node/4803/', 'http://transcript.duma.gov.ru/node/4730/',
         'http://transcript.duma.gov.ru/node/4747/', 'http://transcript.duma.gov.ru/node/4784/', 'http://transcript.duma.gov.ru/node/4787/',
         'http://transcript.duma.gov.ru/node/4791/', 'http://transcript.duma.gov.ru/node/4794/', 'http://transcript.duma.gov.ru/node/4729/',
         'http://transcript.duma.gov.ru/node/4733/', 'http://transcript.duma.gov.ru/node/4738/', 'http://transcript.duma.gov.ru/node/4741/',
         'http://transcript.duma.gov.ru/node/4744/', 'http://transcript.duma.gov.ru/node/4753/', 'http://transcript.duma.gov.ru/node/4754/',
         'http://transcript.duma.gov.ru/node/4757/', 'http://transcript.duma.gov.ru/node/4763/', 'http://transcript.duma.gov.ru/node/4772/',
         'http://transcript.duma.gov.ru/node/4775/', 'http://transcript.duma.gov.ru/node/4783/', 'http://transcript.duma.gov.ru/node/4797/',
         'http://transcript.duma.gov.ru/node/4800/', 'http://transcript.duma.gov.ru/node/4807/', 'http://transcript.duma.gov.ru/node/4810/',
         'http://transcript.duma.gov.ru/node/4813/', 'http://transcript.duma.gov.ru/node/4718/', 'http://transcript.duma.gov.ru/node/4735/']

In [17]:
#4-ая сессия ГД седьмого созыва (весна 2018 г.)
urls_3 = ['http://transcript.duma.gov.ru/node/4892/', 'http://transcript.duma.gov.ru/node/4922/', 'http://transcript.duma.gov.ru/node/4928/',
         'http://transcript.duma.gov.ru/node/4816/', 'http://transcript.duma.gov.ru/node/4819/', 'http://transcript.duma.gov.ru/node/4840/',
         'http://transcript.duma.gov.ru/node/4949/', 'http://transcript.duma.gov.ru/node/4955/', 'http://transcript.duma.gov.ru/node/4977/',
         'http://transcript.duma.gov.ru/node/4837/', 'http://transcript.duma.gov.ru/node/4846/', 'http://transcript.duma.gov.ru/node/4855/',
         'http://transcript.duma.gov.ru/node/4858/', 'http://transcript.duma.gov.ru/node/4881/', 'http://transcript.duma.gov.ru/node/4884/',
         'http://transcript.duma.gov.ru/node/4886/', 'http://transcript.duma.gov.ru/node/4898/', 'http://transcript.duma.gov.ru/node/4901/',
         'http://transcript.duma.gov.ru/node/4904/', 'http://transcript.duma.gov.ru/node/4907/', 'http://transcript.duma.gov.ru/node/4910/',
         'http://transcript.duma.gov.ru/node/4916/', 'http://transcript.duma.gov.ru/node/4925/', 'http://transcript.duma.gov.ru/node/4934/',
         'http://transcript.duma.gov.ru/node/4952/', 'http://transcript.duma.gov.ru/node/4822/', 'http://transcript.duma.gov.ru/node/4825/',
         'http://transcript.duma.gov.ru/node/4828/', 'http://transcript.duma.gov.ru/node/4831/', 'http://transcript.duma.gov.ru/node/4834/']

In [18]:
#5-ая сессия ГД седьмого созыва (осень 2018 г.)
urls_4 = ['http://transcript.duma.gov.ru/node/5054/', 'http://transcript.duma.gov.ru/node/5038/', 'http://transcript.duma.gov.ru/node/4995/',
         'http://transcript.duma.gov.ru/node/5026/', 'http://transcript.duma.gov.ru/node/5048/', 'http://transcript.duma.gov.ru/node/4989/',
         'http://transcript.duma.gov.ru/node/5001/', 'http://transcript.duma.gov.ru/node/5005/', 'http://transcript.duma.gov.ru/node/5008/',
         'http://transcript.duma.gov.ru/node/5014/', 'http://transcript.duma.gov.ru/node/5045/', 'http://transcript.duma.gov.ru/node/5051/',
         'http://transcript.duma.gov.ru/node/5063/', 'http://transcript.duma.gov.ru/node/5075/', 'http://transcript.duma.gov.ru/node/4962/',
         'http://transcript.duma.gov.ru/node/4965/', 'http://transcript.duma.gov.ru/node/4983/', 'http://transcript.duma.gov.ru/node/4992/',
         'http://transcript.duma.gov.ru/node/5011/', 'http://transcript.duma.gov.ru/node/5017/', 'http://transcript.duma.gov.ru/node/5020/',
         'http://transcript.duma.gov.ru/node/5023/', 'http://transcript.duma.gov.ru/node/5057/', 'http://transcript.duma.gov.ru/node/5078/',
         'http://transcript.duma.gov.ru/node/4968/', 'http://transcript.duma.gov.ru/node/4971/', 'http://transcript.duma.gov.ru/node/4986/',
         'http://transcript.duma.gov.ru/node/5029/', 'http://transcript.duma.gov.ru/node/5035/', 'http://transcript.duma.gov.ru/node/5042/',
         'http://transcript.duma.gov.ru/node/5060/', 'http://transcript.duma.gov.ru/node/5066/', 'http://transcript.duma.gov.ru/node/5069/',
         'http://transcript.duma.gov.ru/node/5072/', 'http://transcript.duma.gov.ru/node/5032/']

In [19]:
#6-ая сессия ГД седьмого созыва (весна 2019 г.)
urls_5 = ['http://transcript.duma.gov.ru/node/5186/', 'http://transcript.duma.gov.ru/node/5112/', 'http://transcript.duma.gov.ru/node/5167/',
         'http://transcript.duma.gov.ru/node/5183/', 'http://transcript.duma.gov.ru/node/5211/', 'http://transcript.duma.gov.ru/node/5217/',
         'http://transcript.duma.gov.ru/node/5115/', 'http://transcript.duma.gov.ru/node/5235/', 'http://transcript.duma.gov.ru/node/5176/',
         'http://transcript.duma.gov.ru/node/5198/', 'http://transcript.duma.gov.ru/node/5201/', 'http://transcript.duma.gov.ru/node/5238/',
         'http://transcript.duma.gov.ru/node/5241/', 'http://transcript.duma.gov.ru/node/5082/', 'http://transcript.duma.gov.ru/node/5091/',
         'http://transcript.duma.gov.ru/node/5103/', 'http://transcript.duma.gov.ru/node/5145/', 'http://transcript.duma.gov.ru/node/5189/',
         'http://transcript.duma.gov.ru/node/5195/', 'http://transcript.duma.gov.ru/node/5205/', 'http://transcript.duma.gov.ru/node/5214/',
         'http://transcript.duma.gov.ru/node/5224/', 'http://transcript.duma.gov.ru/node/5244/', 'http://transcript.duma.gov.ru/node/5253/',
         'http://transcript.duma.gov.ru/node/5094/', 'http://transcript.duma.gov.ru/node/5097/', 'http://transcript.duma.gov.ru/node/5118/',
         'http://transcript.duma.gov.ru/node/5127/', 'http://transcript.duma.gov.ru/node/5130/', 'http://transcript.duma.gov.ru/node/5139/',
         'http://transcript.duma.gov.ru/node/5151/', 'http://transcript.duma.gov.ru/node/5157/', 'http://transcript.duma.gov.ru/node/5158/',
         'http://transcript.duma.gov.ru/node/5164/', 'http://transcript.duma.gov.ru/node/5173/', 'http://transcript.duma.gov.ru/node/5179/',
         'http://transcript.duma.gov.ru/node/5208/', 'http://transcript.duma.gov.ru/node/5223/', 'http://transcript.duma.gov.ru/node/5229/',
         'http://transcript.duma.gov.ru/node/5250/', 'http://transcript.duma.gov.ru/node/5085/', 'http://transcript.duma.gov.ru/node/5088/',
         'http://transcript.duma.gov.ru/node/5100/', 'http://transcript.duma.gov.ru/node/5106/', 'http://transcript.duma.gov.ru/node/5109/',
         'http://transcript.duma.gov.ru/node/5124/', 'http://transcript.duma.gov.ru/node/5136/', 'http://transcript.duma.gov.ru/node/5148/',
         'http://transcript.duma.gov.ru/node/5161/', 'http://transcript.duma.gov.ru/node/5170/', 'http://transcript.duma.gov.ru/node/5192/',
         'http://transcript.duma.gov.ru/node/5220/', 'http://transcript.duma.gov.ru/node/5232/', 'http://transcript.duma.gov.ru/node/5247/',
         'http://transcript.duma.gov.ru/node/5142/']

In [20]:
#7-ая сессия ГД седьмого созыва (осень 2019 г.)
urls_6 = ['http://transcript.duma.gov.ru/node/5288/', 'http://transcript.duma.gov.ru/node/5266/', 'http://transcript.duma.gov.ru/node/5318/',
         'http://transcript.duma.gov.ru/node/5343/', 'http://transcript.duma.gov.ru/node/5315/', 'http://transcript.duma.gov.ru/node/5328/',
         'http://transcript.duma.gov.ru/node/5334/', 'http://transcript.duma.gov.ru/node/5269/', 'http://transcript.duma.gov.ru/node/5275/',
         'http://transcript.duma.gov.ru/node/5291/', 'http://transcript.duma.gov.ru/node/5294/', 'http://transcript.duma.gov.ru/node/5297/',
         'http://transcript.duma.gov.ru/node/5300/', 'http://transcript.duma.gov.ru/node/5337/', 'http://transcript.duma.gov.ru/node/5359/',
         'http://transcript.duma.gov.ru/node/5257/', 'http://transcript.duma.gov.ru/node/5272/', 'http://transcript.duma.gov.ru/node/5278/',
         'http://transcript.duma.gov.ru/node/5281/', 'http://transcript.duma.gov.ru/node/5303/', 'http://transcript.duma.gov.ru/node/5309/',
         'http://transcript.duma.gov.ru/node/5347/', 'http://transcript.duma.gov.ru/node/5350/', 'http://transcript.duma.gov.ru/node/5353/',
         'http://transcript.duma.gov.ru/node/5356/', 'http://transcript.duma.gov.ru/node/5365/', 'http://transcript.duma.gov.ru/node/5260/',
         'http://transcript.duma.gov.ru/node/5263/', 'http://transcript.duma.gov.ru/node/5284/', 'http://transcript.duma.gov.ru/node/5306/',
         'http://transcript.duma.gov.ru/node/5312/', 'http://transcript.duma.gov.ru/node/5321/', 'http://transcript.duma.gov.ru/node/5325/',
         'http://transcript.duma.gov.ru/node/5331/', 'http://transcript.duma.gov.ru/node/5340/', 'http://transcript.duma.gov.ru/node/5362/']

In [21]:
#8-ая сессия ГД седьмого созыва (весна 2020 г.)
urls_7 = ['http://transcript.duma.gov.ru/node/5473/', 'http://transcript.duma.gov.ru/node/5485/', 'http://transcript.duma.gov.ru/node/5491/',
         'http://transcript.duma.gov.ru/node/5391/', 'http://transcript.duma.gov.ru/node/5456/', 'http://transcript.duma.gov.ru/node/5464/',
         'http://transcript.duma.gov.ru/node/5439/', 'http://transcript.duma.gov.ru/node/5378/', 'http://transcript.duma.gov.ru/node/5448/',
         'http://transcript.duma.gov.ru/node/5369/', 'http://transcript.duma.gov.ru/node/5381/', 'http://transcript.duma.gov.ru/node/5400/',
         'http://transcript.duma.gov.ru/node/5403/', 'http://transcript.duma.gov.ru/node/5406/', 'http://transcript.duma.gov.ru/node/5427/',
         'http://transcript.duma.gov.ru/node/5431/', 'http://transcript.duma.gov.ru/node/5445/', 'http://transcript.duma.gov.ru/node/5457/',
         'http://transcript.duma.gov.ru/node/5476/', 'http://transcript.duma.gov.ru/node/5482/', 'http://transcript.duma.gov.ru/node/5375/',
         'http://transcript.duma.gov.ru/node/5397/', 'http://transcript.duma.gov.ru/node/5409/', 'http://transcript.duma.gov.ru/node/5415/',
         'http://transcript.duma.gov.ru/node/5418/', 'http://transcript.duma.gov.ru/node/5430/', 'http://transcript.duma.gov.ru/node/5452/',
         'http://transcript.duma.gov.ru/node/5463/', 'http://transcript.duma.gov.ru/node/5468/', 'http://transcript.duma.gov.ru/node/5488/',
         'http://transcript.duma.gov.ru/node/5494/', 'http://transcript.duma.gov.ru/node/5497/', 'http://transcript.duma.gov.ru/node/5372/',
         'http://transcript.duma.gov.ru/node/5384/', 'http://transcript.duma.gov.ru/node/5387/', 'http://transcript.duma.gov.ru/node/5394/',
         'http://transcript.duma.gov.ru/node/5412/', 'http://transcript.duma.gov.ru/node/5434/', 'http://transcript.duma.gov.ru/node/5442/',
         'http://transcript.duma.gov.ru/node/5472/', 'http://transcript.duma.gov.ru/node/5479/', 'http://transcript.duma.gov.ru/node/5500/']

In [22]:
#9-ая сессия ГД седьмого созыва (осень 2020 г.)
urls_8 = ['http://transcript.duma.gov.ru/node/5511/', 'http://transcript.duma.gov.ru/node/5523/', 'http://transcript.duma.gov.ru/node/5534/',
         'http://transcript.duma.gov.ru/node/5518/', 'http://transcript.duma.gov.ru/node/5527/', 'http://transcript.duma.gov.ru/node/5539/',
         'http://transcript.duma.gov.ru/node/5504/', 'http://transcript.duma.gov.ru/node/5514/', 'http://transcript.duma.gov.ru/node/5519/',
         'http://transcript.duma.gov.ru/node/5529/', 'http://transcript.duma.gov.ru/node/5531/', 'http://transcript.duma.gov.ru/node/5544/',
         'http://transcript.duma.gov.ru/node/5547/', 'http://transcript.duma.gov.ru/node/5550/', 'http://transcript.duma.gov.ru/node/5553/',
         'http://transcript.duma.gov.ru/node/5558/', 'http://transcript.duma.gov.ru/node/5561/', 'http://transcript.duma.gov.ru/node/5564/',
         'http://transcript.duma.gov.ru/node/5570/', 'http://transcript.duma.gov.ru/node/5577/', 'http://transcript.duma.gov.ru/node/5541/',
         'http://transcript.duma.gov.ru/node/5557/', 'http://transcript.duma.gov.ru/node/5567/', 'http://transcript.duma.gov.ru/node/5573/',
         'http://transcript.duma.gov.ru/node/5507/']

In [23]:
#10-ая сессия ГД седьмого созыва (весна 2021 г.)
urls_9 = ['http://transcript.duma.gov.ru/node/5615/', 'http://transcript.duma.gov.ru/node/5589/', 'http://transcript.duma.gov.ru/node/5610/',
         'http://transcript.duma.gov.ru/node/5617/', 'http://transcript.duma.gov.ru/node/5630/', 'http://transcript.duma.gov.ru/node/5636/',
         'http://transcript.duma.gov.ru/node/5670/', 'http://transcript.duma.gov.ru/node/5683/', 'http://transcript.duma.gov.ru/node/5587/',
         'http://transcript.duma.gov.ru/node/5604/', 'http://transcript.duma.gov.ru/node/5621/', 'http://transcript.duma.gov.ru/node/5641/',
         'http://transcript.duma.gov.ru/node/5649/', 'http://transcript.duma.gov.ru/node/5655/', 'http://transcript.duma.gov.ru/node/5661/',
         'http://transcript.duma.gov.ru/node/5674/', 'http://transcript.duma.gov.ru/node/5580/', 'http://transcript.duma.gov.ru/node/5583/',
         'http://transcript.duma.gov.ru/node/5595/', 'http://transcript.duma.gov.ru/node/5598/', 'http://transcript.duma.gov.ru/node/5599/',
         'http://transcript.duma.gov.ru/node/5608/', 'http://transcript.duma.gov.ru/node/5623/', 'http://transcript.duma.gov.ru/node/5624/',
         'http://transcript.duma.gov.ru/node/5629/', 'http://transcript.duma.gov.ru/node/5635/', 'http://transcript.duma.gov.ru/node/5642/',
         'http://transcript.duma.gov.ru/node/5648/', 'http://transcript.duma.gov.ru/node/5654/', 'http://transcript.duma.gov.ru/node/5671/',
         'http://transcript.duma.gov.ru/node/5677/', 'http://transcript.duma.gov.ru/node/5680/']

In [24]:
urls_all =[*urls_0, *urls_1, *urls_2, *urls_3, *urls_4, *urls_5, *urls_6, *urls_7, *urls_8, *urls_9]

In [25]:
for url in urls_all:
    get_words(url)